In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pickle
import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn_gbmi import *

from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, plot_confusion_matrix, make_scorer
from sklearn.inspection import plot_partial_dependence
from sklearn.inspection import partial_dependence
from sklearn.metrics import roc_auc_score

# Importing the data

The data is in a SPSS supported 'SAV' file. We will import it using pandas library's `read_spss` module.

In [ ]:
# Importing SPSS file with pandas
raw = pd.read_spss("DESTINATION.sav")
print(f"Number of samples: {raw.shape[0]}")
print(f"Number of features: {raw.shape[1]}")

# Inspecting data

## Feature types
It is important to inspect the column names, data type, and non-null count. For this dataset, there are 45415 samples and 66 columns in the raw data. All columns are of `floar64` type except one. However, there are multiple columns which are of categorical type or are encoded with one-hot code system and have been put as dummy variable. `mode1` is the response variable which is the target of this analysis. `Consolidat` column is the trip type defined by `1, 3, 2, 7, 5, 6, and 4` values. 1 is the home-to-work trip based on which the dataset will be filtered.

In [ ]:
raw.info()

# Data Pre-processing

Before starting analysis, the dataset needs to be pre-processed so that we can use it to build our classifier models. The steps of pre-processing would be:
1. Filter the sample trips in order to keep only the home-to-work trips.
2. Drop a number of columns keeping only the relevant features.
3. Rename the columns for convenience.
4. Add a `low_income` column as a dummy variable.
5. Convert the distances from meters to km.
6. Check for missing data.
7. Check for anomalies and delete samples with anomalous data.
8. Split the columns to numerical and categorical columns.
9. Scale numeric features.
10. Convert coded columns from float to integer.

After the processing steps, the final dataset will be ready to be put through classifier models.

## Steps 1-5

In [ ]:
# Copying the raw data to keep the original
data = raw.copy(deep = True)

# Selecting 'Consolida = 1', meaning only Home-to-Work trip
data = data.loc[data['Consolidat'] == 1]

# Selecting unecessary and repetative columns to drop
cols_to_drop = ['Member_Cod', 'Origin_TAZ', 'Destinatio', 'Purpose', 'Consolidat', 'Mode', 'RESIDENCE_taz',
                'Age_class', 'Adolescenc', 'Young', 'Adult', 'Senior_Adu', 'Employment', 'Worker', 'Student',
                'Housewife', 'Househol_iNCOME', 'Househol_m', 'Househol_c', 'BINNED_JHB', 'job_rich', 'BINED_HH_I',
                'ACTIVE', 'PUBLIC', 'BINNED_DTT', 'medium_DTT', 'LONGER_DTT', 'BINNED_PTA', 'MPTA',
                'HPTA', 'Ward_old', 'PTA_D', 'DTT_R_KM', 'DTT_D_KM', 'POPU_R_ACRE', 'POPU_D_ACRE', 'EMPLOY_D_ACRE',
                'EMPLOY_R_ACRE',  'Access_fac', 'transit_ac', 'balanced', 'Personal_I', 'DISTANCE']

# Renaming the columns
cols = ['id', 'travel_time', 'household_size', 'bicycle_own', 'gender', 'age', 'driving_license',
        'link_node_ratio', 'dist_tran_stop',  'diversity', 'job_hh_ratio', 'pop_density', 'emp_density', 'mode',
        'middle_income', 'high_income', 'car_own', 'dest_link_node_ratio',
        'dest_dist_tran_stop', 'dest_diversity', 'dest_job_hh_ratio', 'dest_pop_density', 'dest_emp_density']

# Drop Columns
data.drop(cols_to_drop, axis = 1, inplace = True)

# Rename columns
data.columns = cols

# Adding 'low_income' category column
data['low_income'] = data.apply(lambda x: 1 if x['middle_income'] == 0 and x['high_income'] == 0 else 0, axis=1)

# Converting distance from meter to kilometer
data.loc[:, ['dist_tran_stop', 'dest_dist_tran_stop']] = data.loc[:, ['dist_tran_stop', 'dest_dist_tran_stop']]/1000

# Checking the data snapshot
data.head(5)

## Step 7: Checking anomalies
### Identifying anomalies with visualization

In [ ]:
sns.scatterplot(data = data, x='dest_link_node_ratio', y = 'travel_time')

### Dropping anomalous data points

In [ ]:
data = data[data.loc[:, 'dest_link_node_ratio'] > 0]

# Converting id to string
data.reset_index(inplace=True, drop=True)
data.loc[:, 'id'] = data.index
data.loc[:, 'id'] = data.loc[:, 'id'].astype('str')

data.head()

In [ ]:
print(f"Number of samples: {data.shape[0]}")
print(f"Number of features: {data.shape[1]}")

## Step 8: Converting feature types
### Splitting categorical and numerical features

In [ ]:
#####
# Coded: 'gender','driving_license', 'low_income', 'middle_income', 'high_income', 'mode' > transform them to integer

# Numerical Columns: 'travel_time', 'household_size', 'age', 'car_own', 'bicycle_own', 'link_node_ratio', 'dist_tran_stop',
# 'diversity', 'job_hh_ratio', 'pop_density', 'emp_density',
# 'dest_link_node_ratio', 'dest_dist_tran_stop', 'dest_diversity', 'dest_job_hh_ratio',
# 'dest_pop_density', 'dest_emp_density'
####
num_cols=['travel_time', 'household_size', 'age', 'car_own', 'bicycle_own', 'link_node_ratio', 'dist_tran_stop',
          'diversity', 'job_hh_ratio', 'pop_density', 'emp_density',
          'dest_link_node_ratio', 'dest_dist_tran_stop', 'dest_diversity', 'dest_job_hh_ratio',
          'dest_pop_density', 'dest_emp_density']

coded = ['gender','driving_license', 'low_income', 'middle_income', 'high_income', 'mode']

## Step 9: Scaling numeric features

The numeric columns have different range. To build a classifier, we need to scale the values so that none of the features get preference from the classifier based on the high range of values.

In [ ]:
df = data.copy(deep = True)
df[coded] = df[coded].astype('int32')
scaler = MinMaxScaler()
scaler.fit(df.loc[:, num_cols])
df.loc[:, num_cols] = scaler.transform(df.loc[:, num_cols])

df = df[num_cols+coded]

# Preparing dataset for classifier

The `mode` feature is our target variable, y. The rest of the features are the X or independent variables. The dataset will be divided into two parts - training and test set. This is known as holdout method, where a certain percentage of data is held out for testing the model performance. We divided the dataset into training and test set in a 70:30 ratio. Thus, 70% of the samples will be used to train the classifiers and the rest of it will be used to test the model performance.

In [ ]:
y = df[['mode']]
X = df.drop(['mode'], axis = 1)
rs = 4

# Splitting dataset into training and test set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=rs)

## Handling unbalanced data

The dataset is a multiclass framework and the classes are `Private, Transit, and Active`. However, the class distribution in the dataset is unbalanced as we have seen before. Meaning, there are a lot more samples for `Transit` and `Active` than `Private`. This issue will negative impact any classifier model built on this dataset. The during the training iterations, the model will learn the characteristics of the `Transit` and `Active` classes better than `Private` class which is not desirable. The goal is to build a model that can predict and give us insight on how the mode choice varies and which features influences the choice. To have an accurate representation, we need the classifier model to be able to train on a balanced dataset. So, we will resolve the issue through over-sampling the minority class. Meaning, we will randomly duplicate the `Private` class samples during training process. This is a well-known solution for resolving issues with unbalanced datasets. Another approach to this issue is to under-sample the dataset. Meaning we try to match the number of samples from each class to the class with lowest occurances. However, this approach reduces the total number of samples drastically and hampers the training process.

In [ ]:
# oversampling the dataset
smote = SMOTE(sampling_strategy='not majority')
# fit and apply the transform
X_over, y_over = smote.fit_resample(X_train, y_train)

In [ ]:
# Random seed
rs = 42

# Model Evaluation Function

In [ ]:
def evaluate_model(clf, grid, data_x, data_y):
    # Define the evaluation procedure
    
    # define the grid search procedure
    grid_search = GridSearchCV(estimator=clf, param_grid=grid, cv=10, scoring='accuracy', verbose = 1,
                               return_train_score=True)
    
    # execute the grid search
    grid_result = grid_search.fit(data_x, data_y)
    
    # summarize the best score and configuration
    print("Best: %.2f using parameters: %s" % (grid_result.best_score_, grid_result.best_params_))
    
    # Performance metric report
    y_pred = grid_result.predict(X_test)
    print(classification_report(y_test, y_pred))
    
    # Plotting confusion matrix
    plt.style.use('classic')
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(121)
    plot_confusion_matrix(grid_result.best_estimator_, data_x, data_y, cmap=plt.cm.Blues, normalize='true', ax=ax1)
    ax2 = fig.add_subplot(122)
    plot_confusion_matrix(grid_result.best_estimator_, X_test, y_test, cmap=plt.cm.Blues, normalize='true', ax=ax2)
    plt.show()
    
    return grid_result

## Evaluation Grid Plotting

In [ ]:
def grid_plot(grid_result):
    train_result = grid_result.cv_results_['mean_train_score']
    test_result = grid_result.cv_results_['mean_test_score']
    
    mpl.rcParams.update(mpl.rcParamsDefault)
    plt.figure(figsize=(8,3))
    #plt.style.use('classic')
    
    plt.plot(range(1, len(train_result)+1), train_result, 'b', label='Training accuracy')
    plt.plot(range(1,len(test_result)+1), test_result, 'r', label='Testing accuracy')
    
    plt.xlabel('Model iterations')
    plt.xticks(range(1, len(train_result)+1, 2), rotation=75)
    plt.ylabel('Score (accuracy)')
    
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

# Parameter grid-search and cross-validation

## Random Forest

In [ ]:
clf_rf = RandomForestClassifier(bootstrap = True, n_jobs = -1, random_state = rs, class_weight = 'balanced')

# Define grid search parameters
grid = {'n_estimators': [100, 500, 1000]
        , 'criterion': ['gini', 'entropy']
        , 'max_depth': list(range(1,10,2))}

# Evaluate the model
grid_result_rf = evaluate_model(clf_rf, grid, X_over, np.ravel(y_over))

grid_plot(grid_result_rf)

## Gradient Boosting

In [ ]:
clf_gbdt = GradientBoostingClassifier(random_state=rs)

# Define grid search parameters
grid = {'n_estimators': [100, 500, 1000]
        , 'learning_rate': [1e-5, 1e-2, 0.1]
        , 'max_depth': [5, 10]
        , 'max_features': ['sqrt', 'log2']}

# Evaluate the model
grid_result_gbdt = evaluate_model(clf_gbdt, grid, X_over, np.ravel(y_over))

grid_plot(grid_result_gbdt)

## Selecting Best Model

In [ ]:
## Best Model: Random Forest
# rf_2 = grid_result_rf.best_estimator_
clf_rf_best = RandomForestClassifier(criterion ='gini', max_depth = 9, n_estimators = 500, bootstrap = True, n_jobs = -1, random_state = rs, class_weight = 'balanced')
# clf_rf_best = RandomForestClassifier(bootstrap = True, n_jobs = -1, random_state = rs, class_weight = 'balanced')
clf_rf_best.fit(X_over, np.array(y_train).ravel())

# Partial Dependance Plot

In [ ]:
def plot_clf_pdp(data_x, feat_list, color, classf, target):
  features = feat_list
  labels = ['Travel time', 'Household Size', 'Age', 'No. of Car Owned', 'No. of Bicycle Owned',
            'Link node ratio', 'Distance to Transit Stop (km)', 'Diversity', 'Job-to-HH ratio (jobs/HH)',
            'Population Density (HH/hectare)', 'Employment Density (jobs/hectare)','Destination Link node ratio',
            'Destination Distance to Transit Stop (km)',
            'Destination Diversity', 'Destination Job-to-HH ratio (jobs/hh)',
            'Destination Population Density (HH/hectare)', 'Destination Employment Density (jobs/hectare)',
            'gender', 'driving_license', 'income']

  mpl.rcParams.update(mpl.rcParamsDefault)
  fig, ax = plt.subplots(figsize=(15,10))
  disp = plot_partial_dependence(classf, data_x, features, target = target, feature_names=labels,
                                        response_method = 'predict_proba',
                                        ax=ax, line_kw={"color": color})
  disp.axes_[0,0].grid(True, color='#dfe4ea')
  disp.axes_[0,1].grid(True, color='#dfe4ea')
  disp.axes_[0,2].grid(True, color='#dfe4ea')
  disp.axes_[1,0].grid(True, color='#dfe4ea')
  disp.axes_[1,1].grid(True, color='#dfe4ea')
  disp.axes_[1,2].grid(True, color='#dfe4ea')

  plt.subplots_adjust(top=0.9)
  plt.show()

  return disp

## Origin

In [ ]:
plot_clf_pdp(X_over, [5,6,7,8,9,10], "#FC427B", clf_rf_best, 1)

## Destination

In [ ]:
plot_clf_pdp(X_over, [11,12,13,14,15,16], "#FC427B", clf_rf_best, 1)

# Dual Partial Dependance Plot

In [ ]:
now = datetime.now()
features =[(9,7),(9,10),(6,8), (15,11),(15,16),(14,16)]
partial = []

for feats in features:
  p = partial_dependence(clf_rf_best, X_over, features=feats, kind='average', grid_resolution=20)
  partial.append(p)

labels = ['Travel time', 'Household Size', 'Age', 'No. of Car Owned', 'No. of Bicycle Owned',
          'Link node ratio', 'Distance to Transit Stop (km)', 'Diversity', 'Job-to-HH ratio (jobs/HH)',
          'Population Density (HH/hectare)', 'Employment Density (jobs/hectare)','Destination Link node ratio',
          'Destination Distance to Transit Stop (km)',
          'Destination Diversity', 'Destination Job-to-HH ratio (jobs/hh)',
          'Destination Population Density (HH/hectare)', 'Destination Employment Density (jobs/hectare)',
          'gender', 'driving_license', 'income']

fig = plt.figure(figsize=(18,12))

for i, p in enumerate(partial):
  ax = fig.add_subplot(2,3,i+1)
  XX, YY = np.meshgrid(p["values"][0], p["values"][1])
  Z = p.average[0].T
  surf = ax.contourf(XX, YY, Z)
  ax.set_xlabel(labels[features[i][0]], fontsize = 15)
  ax.set_ylabel(labels[features[i][1]], fontsize = 15)

  cbar = fig.colorbar(surf, format='%.3f')
  cbar.ax.set_ylabel('Car choice probability', fontsize = 15)

plt.subplots_adjust(top=0.9)
plt.tight_layout()
plt.show()

tot = datetime.now()-now
print(f"Total time to plot: {tot.seconds/60:.2f} miunutes")

# H-Statistics

In [ ]:
pair_h = h_all_pairs(clf_rf_best, X_over)